# MAIN CODE

In [1]:
import requests
import datetime

thunder1=[] #name
thunder2=[] #price
location = []
description = []
time = []
status = []

for page_number in range(0,1):  #125

    url = "https://api.bunjang.co.kr/api/1/find_v2.json?q=%EC%95%84%EC%9D%B4%ED%8F%B011&order=score&page={0}&request_id=2022519113331&stat_device=w&n=100&stat_category_required=1&req_ref=search&version=4".format(page_number)
    res = requests.get(url)
    res.encoding=None
    data = res.json()
    
    for d in data['list']:
        name = d['name']
        loc = d['location']
        price = int(d['price'])
        pid = d['pid']
        unix = str(datetime.datetime.fromtimestamp(d['update_time']))
        unix = unix[0:7]
        sta = d['status']

        
        excepts = ["케이스","삽니다", "매입", "구매", "구입","프로","Pro","pro", "미니", "PRO", "11PRO" , '아이폰7', 'SE', 'se', '부품', 'Se', 'XS', 'max']
        targets = ['배터리', '판매', '팔아요','아이폰11', '아이폰 11']

        if (price >= 50000) & (price < 1600000) ==True :
            if ( d['ad']==False ):
                if any(x in d["name"] for x in targets) and not any(x in d['name'] for x in excepts):
                    thunder1.append(d['name'])
                    thunder2.append(price)
                    time.append(unix)
                    loc = loc.split(' ')
                    loc = loc[0]
                    # 위치
                    if len(loc) == 0:
                        location.append('NULL')
                    else:
                        location.append(loc)
                    # 판매 여부
                    if sta == '0' :           # 0: 판매중    3 : 판매완료
                        status.append('n')  # n 판매중
                    else : status.append('y')  # y 판매완료(예약완료 포함)

                    
                    

                    url2 = 'https://api.bunjang.co.kr/api/1/product/{0}/detail_info.json?version=4'.format(pid)
                    res = requests.get(url2)
                    res.encoding=None
                    data = res.json()
                
                    description.append(data['item_info']['description'])
                    


                    # print(name, price, unix, loc, sta)

        else:
            pass

# print(thunder1,thunder2, time, location )


# Desc 에서 배터리 효율 뽑아내기

In [2]:
import re
from bs4 import BeautifulSoup

li = []



for i in range(len(description)):
    text = description[i]
    regex = '배터리\s*.*\d{2,3}'
    pattern =  re.compile(regex)
    res = pattern.findall(text)
    
    regex2 = '\d{2,3}'
    pattern2 = re.compile(regex2)
    text = str(res)
    res2 = pattern2.findall(text)
    if len(res2) == 0:
        li.append('NULL')
    else:
        li.append(res2[0])

print(li)


['85', 'NULL', 'NULL', '10', 'NULL', 'NULL', '85', '84', '81', 'NULL', '82', '78', 'NULL', '86']


용량 빼내기

In [3]:
size = []


for i in range(len(description)):
    text = description[i]
    text2 = thunder1[i]
    regex = '64|128'
    pattern =  re.compile(regex)
    res = pattern.findall(text)   #상품설명에서 용량 찾기
    res_size = pattern.findall(text2)  #네임에서 용량 찾기
    
    if len(res) == 0:
        if len(res_size) == 0 :
            size.append('NULL')
        else:
            size.append(res_size[0])
    else:
        size.append(res[0])
print(size)

['64', '128', '128', '128', 'NULL', '128', 'NULL', '128', '64', '128', '128', '128', '64', '64']


# 판다스로 데이터프레임 만들기

In [4]:
import pandas as pd
df = pd.DataFrame({'name' : thunder1,
                 'price' : thunder2,
                 'time' : time,
                  'loction' : location,
                  'battery' : li,
                  'size': size,
                  'status' : status
                  })
df

,name,price,time,loction,battery,size,status
0,아이폰 11,400000,2022-05,NULL,85,64,n
1,아이폰11,450000,2022-05,서울특별시,NULL,128,n
2,아이폰11 128기가 블랙 90%,380000,2022-05,강원도,NULL,128,n
3,A급)) 아이폰11 64GB 블랙 팝니다.,400000,2022-05,경기도,10,128,n
4,아이폰 11 256기가,380000,2022-05,부산광역시,NULL,NULL,n
5,아이폰11 레드 128기가,350000,2022-05,서울특별시,NULL,128,n
6,아이폰11 256GB 옐로우 69776 아이폰11 256,430000,2022-05,서울특별시,85,NULL,n
7,아이폰11 128GB A급 화이트 42969,450000,2022-05,서울특별시,84,128,n
8,애플 아이폰11 (A2221) 64G 블랙 B++급 단품,300000,2022-05,부산광역시,81,64,n
9,아이폰 11 128,370000,2022-05,대구광역시,NULL,128,n


# 데이터 엑셀파일로 추출

In [5]:
# from pandas import ExcelWriter
# df.to_excel('C:/Users/admin/Desktop/big/thunder.xlsx', index=False)